In [1]:
import torch
import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/") # PATH TO PYSEQM
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
seqm.seqm_functions.scf_loop.debug=True # print SCF steps 
seqm.seqm_functions.scf_loop.MAX_ITER=500 # MAX number of SCF iterations

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
species = torch.as_tensor([[16,16],[22,22],[22,16],[35,17],[24,22]],dtype=torch.int64, device=device)

coordinates = torch.tensor([
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
                 ], device=device)


const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0, 0.2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                    "Hf_flag": True,
                    'scf_backward' : 0,
                    'UHF' : False, # open shell is currently not supported for PM6
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules)

/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/diag_d.py:109: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2794.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)


scf direct step  :   0 | MAX ΔE[   4]:   81.7746467 | MAX ΔDM[   4]:    0.2552329 | MAX ΔDM_ij[   4]:  1.0000000  | N not converged: 5
scf direct step  :   1 | MAX ΔE[   1]:   10.2522138 | MAX ΔDM[   4]:    0.2373771 | MAX ΔDM_ij[   4]:  1.5960220  | N not converged: 5
scf direct step  :   2 | MAX ΔE[   4]:   23.7740329 | MAX ΔDM[   4]:    0.2111070 | MAX ΔDM_ij[   4]:  1.3746724  | N not converged: 5
scf direct step  :   3 | MAX ΔE[   4]:   12.8381890 | MAX ΔDM[   4]:    0.2170586 | MAX ΔDM_ij[   4]:  1.3256652  | N not converged: 5
scf direct step  :   4 | MAX ΔE[   4]:   16.8776639 | MAX ΔDM[   4]:    0.2015102 | MAX ΔDM_ij[   4]:  1.3941663  | N not converged: 5
scf direct step  :   5 | MAX ΔE[   4]:    7.1436214 | MAX ΔDM[   4]:    0.1931167 | MAX ΔDM_ij[   4]:  1.2433544  | N not converged: 5
scf direct step  :   6 | MAX ΔE[   4]:   11.7135435 | MAX ΔDM[   4]:    0.1793226 | MAX ΔDM_ij[   4]:  1.1463584  | N not converged: 5
scf direct step  :   7 | MAX ΔE[   4]:    1.2483319 | M

# Total E

In [3]:
molecules.Etot[0] # S2. MOPAC Etot = -333.8287 EV

tensor(-333.8298, device='cuda:0')

In [4]:
molecules.Etot[1] # Ti2. MOPAC Etot = -120.0651 EV

tensor(-120.0663, device='cuda:0')

In [5]:
molecules.Etot[2] # TiS. MOPAC Etot = -221.2701 EV

tensor(-221.2703, device='cuda:0')

In [6]:
molecules.Etot[3] # BrCl. MOPAC Etot = -458.5855 EV

tensor(-458.5839, device='cuda:0')

In [7]:
molecules.Etot[4] # CrTi. MOPAC Etot = -150.3163 EV  !!! Wrong nuclear rep parameters

tensor(-255.8133, device='cuda:0')

# Nuclear repulsion

In [8]:
molecules.Enuc[0] # S2. MOPAC Enuc = 290.8693 EV

tensor(290.8621, device='cuda:0')

In [9]:
molecules.Enuc[1] # Ti2. MOPAC Enuc = 97.3923 EV

tensor(97.3923, device='cuda:0')

In [10]:
molecules.Enuc[2] # TiS. MOPAC Enuc = 176.1518 EV

tensor(176.1491, device='cuda:0')

In [11]:
molecules.Enuc[3] # BrCl. MOPAC Enuc = 391.1161 EV

tensor(391.1087, device='cuda:0')

In [12]:
molecules.Enuc[4] # CrTi. MOPAC Enuc = 251.0941 EV  !!! Wrong nuclear rep parameters

tensor(145.6016, device='cuda:0')